# Partitioning

In [0]:
from pyspark.storagelevel import StorageLevel
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [0]:
%run ../PySpark/DatasetSourcePath

In [0]:
listening_activity_file = sourcePath + "/dataset/partitioning/Spotify_Listening_Activity.csv"
df_listening_actv = spark.read.csv(listening_activity_file, header=True, inferSchema=True)
df_listening_actv.show(5, False)

In [0]:
df_listening_actv = (
    df_listening_actv
    .withColumnRenamed("listen_date", "listen_time")
    .withColumn("listen_date", to_date("listen_time", "yyyy-MM-dd HH:mm:ss.SSSSSS"))
    .withColumn("listen_hour", hour("listen_time"))
)

df_listening_actv.show(5, False)
df_listening_actv.printSchema()
df_listening_actv.count()

## Partitioning By `listen_date`

Let's say we want to **analyse the listening behaviours of user over time**. If we're given the complete dataset (with no partitions), Spark would scan the whole dataset for finding a particular date (similar to the bookshelf analogy where you would scan the entire bookself for finding a book if it is not organized). Given that our usecase needs analysis by date, partitioning (creating folders) on date would help Spark pin point to the exact folder. This makes searching very easy and Spark doesn't scan the entire dataset.  

In [0]:
# Partitioning listening activity by the listen date
(
    df_listening_actv
    .write
    .partitionBy("listen_date")
    .mode("overwrite")
    .parquet("/tmp/partitioning/listening_activity_pt")
)

## Partition Pruning

In [0]:
df_listening_actv_pt_pruned = spark.read.parquet("/tmp/partitioning/listening_activity_pt")
df_listening_actv_pt_pruned.filter("listen_date = '2019-01-01'").explain()

## What Problems Does Partitioning Solve? 
1. `Fast Search (Query Performance)`: Spark will only process the relevant partition instead of the entire dataset (example above). This greatly reduces I/O and query execution time. 
2. `Parallelism / Resource Utilization`: Each core processes 1 partition; More number of partitions, more is the parallelism; again this does not mean we forcefully increase the number of partitions. Each partition should be `128MB` in size. 


# Partitioning Examples
1. Single/multi level partitioning
2. Using `repartition`/`coalesce` with `partitionBy` (controlling number of files inside each partition): 
    - `parititionBy` affects how data is laid out in the storage and is going to ensure that the output directory is organized into subdirectories based on the `value` given in `partitionBy`.  
    - Number of files in each `value` directory of `partitionBy` depends on the number supplied in the `repartition`/`coalesce`.

#### 1. Single/multi level partitioning

In [0]:
df_listening_actv.printSchema()

In [0]:
(
    df_listening_actv
    .write
    .mode("overwrite")
    .partitionBy("listen_date", "listen_hour")
    .parquet("/tmp/partitioning/listening_activity_pt_2")
)

In [0]:
(
    df_listening_actv
    .write
    .mode("overwrite")
    .partitionBy("listen_hour", "listen_date")
    .parquet("/tmp/partitioning/listening_activity_pt_2")
)

#### 2. Using `repartition`/`coalesce` with `partitionBy`

In [0]:
df_listening_actv.rdd.getNumPartitions()

In [0]:
(
    df_listening_actv
    .repartition(3)
    .write
    .mode("overwrite")
    .parquet("/tmp/partitioning/listening_activity_pt_4")
)

In [0]:
# The coalesce method reduces the number of partitions in a DataFrame. 
# It avoids full shuffle, instead of creating new partitions, it shuffles the data using default Hash Partitioner, 
# and adjusts into existing partitions, this means it can only decrease the number of partitions.

(
    df_listening_actv
    .coalesce(5)
    .write
    .mode("overwrite")
    .parquet("/tmp/partitioning/listening_activity_pt_5")
)

## Experimenting With `spark.sql.files.maxPartitionBytes`

In [0]:
spark.conf.get("spark.sql.files.maxPartitionBytes")


In [0]:
path = sourcePath + "/dataset/partitioning/Spotify_Listening_Activity.csv"
df_default = spark.read.csv(path, header=True, inferSchema=True)
default_partitions = df_default.rdd.getNumPartitions()
print(f"Number of partitions: {default_partitions}")


In [0]:
spark.conf.set("spark.sql.files.maxPartitionBytes", "1000")

In [0]:
path = sourcePath + "/dataset/partitioning/Spotify_Listening_Activity.csv"
df_modified = spark.read.csv(path, header=True, inferSchema=True)

modified_partitions = df_modified.rdd.getNumPartitions()
print(f"Number of partitions: {modified_partitions}")